## 数据预处理

创建一个人工数据集，并存储在csv(逗号分割值)文件（注意：并不是读一个现成的数据文件，而是自己创造一个文件）

In [3]:
import os
os.makedirs(os.path.join("..","data"),exist_ok=True)
data_file=os.path.join("..","data","house_tiny.csv")

In [4]:
with open(data_file,"w") as f:
    f.write("NumRooms,Alley,Price\n") # 列名
    f.write("NA,Pave,127500\n") # 每行表示一个数据样本
    f.write("2,NA,10600\n") 
    f.write("4,NA,178100\n")
    f.write("NA,NA,140000\n")

从创建的csv文件中加载原始数据集

In [5]:
import pandas as pd
data=pd.read_csv(data_file)

In [6]:
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,10600
2,4.0,NaN,178100
3,NaN,NaN,140000


为了处理缺失的数据，典型方法包括插值和删除，这里，考虑使用插值

+ 处理数据的时候，分成输入的特征和输出的预测结果（这里的demo中价格是最终要预测的内容）
+ 这里使用平均值填充输入中缺失的数据（数值部分可以进行这样的平均值填充，非数值对象则不能）
+ fillna() Fill NA/NaN values using the specified method.

In [31]:
inputs,outputs=data.iloc[:,0:2],data.iloc[:,2]
inputs=inputs.fillna(inputs.mean())

In [32]:
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


**另外，对于`inputs`中的类别值或离散值，可以将`NaN`视为一个类别**

即对于上面的Alley这一列，一共有两种值，Pave和NaN，此时可以将这两种值作为两种类别。

In [33]:
inputs=pd.get_dummies(inputs,dummy_na=True)
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


至此，`inputs`和`outputs`中所有条目都是数值类型，可以转换为张量格式

In [35]:
inputs.values

array([[3., 1., 0.],
       [2., 0., 1.],
       [4., 0., 1.],
       [3., 0., 1.]])

In [34]:
type(inputs.values)

numpy.ndarray

In [36]:
import torch
X,y=torch.tensor(inputs.values),torch.tensor(outputs.values)
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500,  10600, 178100, 140000]))

**可以看到，python一般会默认是64位浮点数，但是64位浮点数一般计算会比较慢，对深度学习来讲，一般用32位浮点数**

## 插曲——缺失值填充

In [10]:
import numpy as np

In [16]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],[3, 4, np.nan, 1],[np.nan, np.nan, np.nan, 5],[np.nan, 3, np.nan, 4]],columns=list('ABCD'))

In [17]:
df.fillna(value=df.mean())

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,3.0,NaN,5
3,3.0,3.0,NaN,4


In [24]:
values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

In [25]:
df.fillna(value=values)

,A,B,C,D
0,0.0,2.0,2.0,0
1,3.0,4.0,2.0,1
2,0.0,1.0,2.0,5
3,0.0,3.0,2.0,4


In [21]:
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,3.0,NaN,5
3,NaN,3.0,NaN,4
